# Data 살펴보기위한 곳

In [5]:
import os
import json
import pandas as pd

os.chdir('/root/')

PATH = "./data/"
with open(PATH+'preprocessed_data.json','r') as file:
            data = json.load(file)

In [7]:
labels = {}
count = 0
for item in data['data']:
    count += 1
    emo_dic = {}
    elem_list = []
    for emo in item['Emotion']:
        emo_dic[emo] = emo_dic.get(emo, 0) + 1
    for k, v in emo_dic.items():
        elem_list.append([v, k])
    elem_list.sort()
    top_emo = elem_list.pop()
    '''
    if elem_list:
        if(top_emo[0] == elem_list[-1][0]):
            while(elem_list and top_emo[0] == elem_list[-1][0]):
                print(top_emo, elem_list[-1])
                if (top_emo[1] in ['neutral', 'angry', 'sad']):
                    top_emo = elem_list.pop()
                else:
                    break
    '''
    #else:
     #   labels['only_'+top_emo[1]] = labels.get('only_'+top_emo[1], 0) + 1
    
    labels[top_emo[1]] = labels.get(top_emo[1], 0) + 1
    
print(labels)
print(count)

{'neutral': 6061, 'happy': 5808, 'sad': 6344, 'angry': 6000, 'surprise': 4256, 'fear': 4984, 'disgust': 4513}
37966


# 감정분석 데이터 가져오기 테스트

In [ ]:
import pandas as pd
import shutil
import json
import os

import numpy as np

def GenerateSubDir(PATH, COPYPATH):
    # 파일 복사 (현재 경로, 복사할 경로)
    def CopyFile(cur_path, COPYPATH):
        if not (os.path.isfile(COPYPATH)):
            shutil.copy(cur_path, COPYPATH)
        
    # 재귀적으로 파일 탐색(현재 경로, 복사할 경로, 파일의 태그(종류, wav, EDA ....))
    def SearchFiles(path, COPYPATH, tagname):
        for cur in os.listdir(path):
            cur_path = os.path.join(path, cur)
            if os.path.isdir(cur_path):
                SearchFiles(cur_path, COPYPATH, tagname)
            else:
                CopyFile(cur_path, os.path.join(COPYPATH, tagname+'_'+cur))

    os.makedirs(COPYPATH, exist_ok=True)
    for Ori in ["감정 분류를 위한 대화 음성 데이터셋"]:
        for cur in os.listdir(os.path.join(PATH, Ori)):
            cur_path = os.path.join(PATH, Ori, cur)
            if os.path.isdir(cur_path):
                SearchFiles(cur_path, COPYPATH, cur)
            else:
                CopyFile(cur_path, os.path.join(COPYPATH, cur))

def Setting_Emotion(series):
    encoder = {
        'neutral' : 'neutral',
        'angry' : 'angry',
        'sadness' : 'sad',
        'disgust' : 'disgust',
        'happiness' : 'happy',
        'fear' : 'fear',
        'surprise' : 'surprise'
    }
    data = series.values[3:]
    emotions = []

    for i in range(0, len(data), 2):
        emo = data[i].lower()
        emo = encoder[emo]
        count = data[i+1]+1
        emotions += [emo]*count
    return emotions
    
def Read_DataFrames(COPYPATH, file_name):
        df = pd.read_csv(os.path.join(COPYPATH, file_name), encoding="cp949")
        df = df[df.columns[:-2]]

        emotions = []
        wavid = []
        for i in range(len(df)):
            emotions.append(Setting_Emotion(df.iloc[i]))
            wavid.append(file_name[:-4]+ '_' + df['wav_id'].iloc[i]+".wav")
        df['Emotion'] = emotions
        df['wav'] = wavid
        df = df[['wav_id', 'wav', '발화문', 'Emotion']]
        df.columns = ['file_name', 'wav', 'utterance', 'Emotion']
        return df

In [ ]:
os.chdir("/root/")
#if __name__ == '__main__':
# Move All files to one directory
PATH = './'
COPYPATH = os.path.join(PATH, "TOTAL")
#GenerateSubDir(PATH, COPYPATH)


# 아까 모아놓았던 경로
base_df = pd.DataFrame(columns=['file_name', 'wav', 'utterance', 'Emotion'])

for file_name in ['4차년도.csv','5차년도.csv', '5차년도_2차.csv']:
    df = Read_DataFrames(COPYPATH, file_name)
    base_df = pd.concat([base_df, df], axis=0)

json_path = os.path.join(PATH, 'data', 'processed_data.json')
with open(json_path,'r') as file:
    data = json.load(file)

for i in range(len(base_df)):
    item = {key:val for key, val in zip(base_df.columns, base_df.iloc[i].values)}
    data['data'].append(item)

data

KeyError: 'data'

# MP4 to wav

In [1]:
import os

def ExtractWav(path, cur):
    file_name, ext = os.path.splitext(cur)

    if(ext == ".m2ts"):
        audio = mp.AudioFileClip(os.path.join(path, cur))
        audio.write_audiofile(os.path.join(path, file_name+".wav"))

        arr = audio.to_soundarray()
    
        
    # 재귀적으로 파일 탐색(현재 경로, 복사할 경로, 파일의 태그(종류, wav, EDA ....))
def M2ts_to_Wav(path):
    for cur in os.listdir(path):
        cur_path = os.path.join(path, cur)
        if os.path.isdir(cur_path):
            M2ts_to_Wav(cur_path)
        else:
            ExtractWav(path, cur)


import moviepy.editor as mp
PATH = "/root/감정분류용 데이터셋"
M2ts_to_Wav(PATH)


MoviePy - Writing audio in /root/감정분류용 데이터셋/0~9_감정분류_데이터셋/000/movie/000-001.wav


MoviePy - Done.
000-001.wav (420713, 2)
MoviePy - Writing audio in /root/감정분류용 데이터셋/0~9_감정분류_데이터셋/000/movie/000-002.wav


MoviePy - Done.


000-002.wav (375291, 2)
MoviePy - Writing audio in /root/감정분류용 데이터셋/0~9_감정분류_데이터셋/000/movie/000-003.wav


MoviePy - Done.


...

MoviePy - Done.


000-005.wav (397782, 2)
MoviePy - Writing audio in /root/감정분류용 데이터셋/0~9_감정분류_데이터셋/000/movie/000-006.wav


MoviePy - Done.
000-006.wav (464373, 2)
MoviePy - Writing audio in /root/감정분류용 데이터셋/0~9_감정분류_데이터셋/000/movie/000-007.wav


MoviePy - Done.


KeyboardInterrupt: 

# Emotion_classifing

In [14]:
import pandas as pd
import shutil
import json
import os

import numpy as np

def GenerateSubDir(PATH, COPYPATH):
    # 파일 복사 (현재 경로, 복사할 경로)
    def CopyFile(cur_path, COPYPATH):
        if not (os.path.isfile(COPYPATH)):
            file_name, ext = os.path.splitext(cur_path)
            if(ext == '.csv' or ext == '.wav'):
                shutil.copy(cur_path, COPYPATH)
        
    # 재귀적으로 파일 탐색(현재 경로, 복사할 경로, 파일의 태그(종류, wav, EDA ....))
    def SearchFiles(path, COPYPATH, tagname):
        for cur in os.listdir(path):
            cur_path = os.path.join(path, cur)
            if os.path.isdir(cur_path):
                SearchFiles(cur_path, COPYPATH, tagname)
            else:
                CopyFile(cur_path, os.path.join(COPYPATH, tagname+'_'+cur))

    os.makedirs(COPYPATH, exist_ok=True)
    for Ori in ["감정분류용 데이터셋"]:
        for cur in os.listdir(os.path.join(PATH, Ori)):
            cur_path = os.path.join(PATH, Ori, cur)
            if os.path.isdir(cur_path):
                SearchFiles(cur_path, COPYPATH, "감정분류용")
            else:
                CopyFile(cur_path, os.path.join(COPYPATH, cur))

def Setting_Emotion(series):
    encoder = {
        'neutral' : 'neutral',
        'angry' : 'angry',
        'sadness' : 'sad',
        'sad' : 'sad',
        'disgust' : 'disgust',
        'happiness' : 'happy',
        'fear' : 'fear',
        'surprise' : 'surprise'
    }
    data = series.values[3:]
    emotions = []

    for i in range(0, len(data), 2):
        emo = data[i].lower()
        emo = encoder[emo]
        count = data[i+1]+1
        emotions += [emo]*count
    return emotions
    


def Read_DataFrames(COPYPATH, file_name):
    script_df = pd.read_csv(os.path.join(COPYPATH, file_name), encoding="cp949")
    script_df = script_df[script_df.columns[:3]]
    script_df['number'] = script_df['number'].apply(int)
    script_df = script_df.set_index(keys=['number'])

    script_dict = {}
    for i in range(len(script_df)):
        number = int(script_df['number'].iloc[i])
        script = script_df['script'].iloc[i]
        emotion = [script_df['emotion'].iloc[i]]
        script_dict[number] = {
            'script' : script,
            'emotion' : emotion
        }
    
    file_names = []
    for file in os.listdir(COPYPATH):
        file_name, ext = os.path.splitext(file)
        if(file_name[:5] == '감정분류용' and ext == '.wav'):
            file_names.append(file)
    
    data = []
    for item in file_names:
         name, ext = os.path.splitext(item)
         script_key = int(name[-3:])
         if script_dict.get(script_key, 0):
             data.append({
                 'wav' : item,
                 'file_name' : name,
                 'utterance' : script_dict[script_key]['script'],
                 'Emotion' : script_dict[script_key]['scriemotionpt']
             })
    return data

SyntaxError: invalid syntax (3959262471.py, line 94)

In [13]:
os.chdir("/root/")
#if __name__ == '__main__':
# Move All files to one directory
PATH = './'
COPYPATH = os.path.join(PATH, "TOTAL")
#GenerateSubDir(PATH, COPYPATH)


# 아까 모아놓았던 경로
data = pd.DataFrame(columns=['file_name', 'wav', 'utterance', 'Emotion'])

for file_name in ['스크립트.csv']:
    df = Read_DataFrames(COPYPATH, file_name)
    base_df = pd.concat([base_df, df], axis=0)

base_df

,file_name,wav,utterance,Emotion,number,script,emotion
0,NaN,NaN,NaN,NaN,51.0,아이씨. 깜짝이야!.,Surprise
1,NaN,NaN,NaN,NaN,52.0,얼마야 이게 다?.,Surprise
2,NaN,NaN,NaN,NaN,53.0,저걸 혼자 다 먹어?.,Surprise
3,NaN,NaN,NaN,NaN,54.0,왜 이렇게 많이 빠졌지?.,Surprise
4,NaN,NaN,NaN,NaN,55.0,8번에 2번 아니야?.,Surprise
...,...,...,...,...,...,...,...
245,NaN,NaN,NaN,NaN,346.0,오랜만에 가족들 본다고 얼마나 좋았을 거야.,Sad
246,NaN,NaN,NaN,NaN,347.0,이루고 나면 행복할 줄 알았는데 아니더라.,Sad
247,NaN,NaN,NaN,NaN,348.0,할아버지는 만나셨어요?.,Sad
248,NaN,NaN,NaN,NaN,349.0,화부터 내서 죄송합니다.,Sad


# Audio check

In [27]:



import soundfile as sf
import os
os.chdir('/root/')
print(os.getcwd())
print(os.path.isfile('./TOTAL/감정분류용_000-001.wav'))
wav, _ = sf.read('./TOTAL/감정분류용_000-001.wav')
wav.shape[1]

/root
True


2

In [31]:
def ExtractWav(path):
    sound  = mp.AudioFileClip(path)
    return sound

import soundfile as sf
import moviepy.editor as mp
PATH = "/root/감정분류용 데이터셋/0~9_감정분류_데이터셋/000/movie/000-001.wav"
#PATH = "/root/TOTAL/4차년도_5e4942578849ac3aacea545b.wav"
sound = ExtractWav(PATH)
arr = sound.to_soundarray(nbytes=1)
wav, _ = sf.read(PATH)
wav = wav.reshape((1, -1))
wav = wav.squeeze()

print(wav.shape)
#sound.write_audiofile("/root/test.wav", nbytes=1)


(841426,)


In [34]:
import numpy as np
np.array([[1,2],[3,4],[5,6]]).reshape((1, -1)).squeeze()

array([1, 2, 3, 4, 5, 6])

In [25]:
import soundfile as sf
wav, _ = sf.read("/root/TOTAL/4차년도_5e4942578849ac3aacea545b.wav")
wav.shape

(348160,)

In [11]:
wav.shape

(348160,)

In [12]:
wav[0]

0.0

# 